## Working with Polars and Scikit-learn
By the end of this lecture you will be able to:
- use Polars objects to fit Scikit-learn models
- work with categorical columns in a gradient boosting model
- output a Polars `DataFrame` from Sklean pipeline processing tools

You may need to install scikit-learn first

In [1]:
# %pip install scikit-learn

In [2]:
import polars as pl
import numpy as np

from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.ensemble import HistGradientBoostingClassifier,HistGradientBoostingRegressor

from sklearn.metrics import root_mean_squared_error,accuracy_score


In this lecture we fit simple models that aim to predict the binary `Survived` column in the Titanic data with features from a selection of other columns

In [3]:
csv_file = "../data/titanic.csv"

In [4]:
df = (
    pl.read_csv(csv_file)
    .select("Pclass","Sex","Age","Fare","Embarked","Survived")
)
df.head()

Pclass,Sex,Age,Fare,Embarked,Survived
i64,str,f64,f64,str,i64
3,"""male""",22.0,7.25,"""S""",0
1,"""female""",38.0,71.2833,"""C""",1
3,"""female""",26.0,7.925,"""S""",1
1,"""female""",35.0,53.1,"""S""",1
3,"""male""",35.0,8.05,"""S""",0


## Fitting a model with Polars
We can pass a Polars `DataFrame` and `Series` directly to Scikit-learn.

Here we fit a logistic regression model using just the numeric features with some simple `null`-filling

In [5]:
model = LogisticRegression()
# Make the feature matrix and target vector
X = df.select(pl.col("Age","Fare").fill_null(strategy="mean"))
y = df["Survived"]

We can pass this Polars `DataFrame` and `Series` directly to scikit-learn

In [6]:
# Fit the model
model.fit(X,y)

LogisticRegression()

We then make a new `DataFrame` with actual labels and predicted labels

In [7]:
# Make predictions on the training data
pred_df = pl.DataFrame(
    {
        "label":y,
        "pred":model.predict(X)
    }
)

print(f'Accuracy: {100*accuracy_score(pred_df["label"],pred_df["pred"]):.2f}%')

Accuracy: 65.66%


Note that `model.predict(X)` is a Numpy ndarray that we turn into a column in a Polars `DataFrame`

In [8]:
type(model.predict(X))

numpy.ndarray

## Categorical features in a gradient boosting model
Typically we have to encode categorical features manually (we see how to do this in pipelines later in this lecture). However, if we use Scikit-learn's gradient boosting model `HistGradientBoostingClassifier` (or `HistGradientBoostingRegressor`) we can pass a Polars `pl.Categorical` column directly to the model without encoding it.

In this example we cast the integer `Pclass` and string `Sex` column to categorical. Recall that to cast an integer column to categorical we must first cast to `pl.String`.

We can pass a Polars `DataFrame` directly to `train_test_split`

In [9]:
df_train, df_test = train_test_split(
    df.select("Pclass","Age","Fare","Sex","Survived"),
    test_size=0.2, 
    random_state=0
)

We then create our feature matrix `X` and target vector `y` by:
- casting `Pclass` and `Sex` to categorical
- filling `nulls` in the `Age` column

In [10]:
X_train = (
    df_train
    .select(
        pl.col("Pclass").cast(pl.String).cast(pl.Categorical),
        pl.col("Sex").cast(pl.Categorical),
        pl.col("Age").fill_null(pl.col("Age").median()),
        pl.col("Fare")
    )
)
y_train = df_train["Survived"]

We can now fit the model and make predic pass the categorical columns directly by passing the `categorical_features="from_dtype"` argument to the model

In [11]:
model_grad_boost = HistGradientBoostingClassifier(categorical_features="from_dtype")

model_grad_boost.fit(X_train,y_train)

HistGradientBoostingClassifier(categorical_features='from_dtype')

Now we make the test feature matrix.

We use `with_context` to fill `nulls` in the `Age` column of the test data with the median of the training data. We do this by:
- `converting `df_test` to lazy mode
- in `with_context` reference `df_train` and get the fill value as a column called `Age_median`
- filling `nulls` in `Age` with the `Age_median` expression

In [12]:
X_test = (
    df_test
    .lazy()
    .with_context(
        df_train
        .lazy()
        .select(pl.col("Age").median().alias("Age_median"))
    )
    .select(
        pl.col("Pclass").cast(pl.String).cast(pl.Categorical),
        pl.col("Sex").cast(pl.Categorical),
        pl.col("Age").fill_null(pl.col("Age_median")),
        pl.col("Fare")
    )
    .collect()
)
y_test = df_test["Survived"]

We now make predictions on the test data and check the accuracy

In [13]:
pred_df = pl.DataFrame(
    {
        "label":y_test,
        "pred":model_grad_boost.predict(X_test)
    }
)
print(f'Accuracy: {100*accuracy_score(pred_df["label"],pred_df["pred"]):.2f}%')

Accuracy: 85.47%


## Scikit-learn pipelines
A more systematic way to produce ML pipelines is to use Scikit-learn's preprocessing tools: 
- `Pipeline` to compose multiple transformation steps on a column together
- `ColumnTransformer` to run transformations on multiple columns together
- `Pipeline` (again) to compose the pre-proprocessing and model fit/predict steps together

In this example we do not cast the categorical columns to `pl.Categorical` but instead use sklearn preprocessing to create categorical features inside the preprocessing objects

In [14]:
df_train, df_test = train_test_split(
    df.select("Pclass","Age","Fare","Sex","Survived"),
    test_size=0.2, 
    random_state=0
)
X_train,y_train = df_train.drop("Survived"),df_train["Survived"]
X_test,y_test = df_test.drop("Survived"),df_test["Survived"]

We first create the `Pipeline` for the columns with numerical features. In this example we:
- use `SimpleImputer` to fill nulls with the median value and
- `StandardScaler` to convert numerical columns to their z-score (i.e. subtracting the mean and dividing by the standard deviation)

In [15]:
numeric_features = ["Age", "Fare"]
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")), 
        ("scaler", StandardScaler())
    ]
)

We then create a one-hot encoding pipeline for categorical features. Note that we have to set `sparse_output=False` if we want the output as a Polars `DataFrame`. If you have a lot of sparse data you may be better off using the normal sparse matrix as the output and not using Polars in the pipeline

In [16]:
categorical_features = ["Sex", "Pclass"]
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore",sparse_output=False)),
    ]
)

We then make a `ColumnTransformer` to handle the preprocessing for the different types of columns

In [17]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ],
)

We create another `Pipeline` to handle preprocessing and model fitting. In this example we set the output to be Polars objects by calling `set_output` on the `Pipeline`

In [18]:
preprocess_model_pipeline = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)
preprocess_model_pipeline.set_output(transform="polars")

preprocess_model_pipeline.fit(X_train, y_train)
print("model score: %.3f" % preprocess_model_pipeline.score(X_test, y_test))

pred_df = pl.DataFrame(
    {
        "label":y_test,
        "pred":preprocess_model_pipeline.predict(X_test)
    }
)
pred_df

model score: 0.799


label,pred
i64,i64
0,0
0,0
0,0
1,1
1,1
…,…
1,1
0,0
1,1


## Exercises

### Exercise one
We want to predict the tip amount in the NYC taxi database from the other columns

In [19]:
taxi_data = "../data/nyc_trip_data_1k.csv"
df_nyc = pl.read_csv(taxi_data,try_parse_dates=True)
df_nyc

VendorID,pickup,dropoff,passenger_count,trip_distance,fare_amount,tip_amount
str,datetime[μs],datetime[μs],f64,f64,f64,f64
"""id1""",2022-01-01 00:04:14,2022-01-01 00:26:12,1.0,10.83,31.0,0.0
"""id2""",2022-01-01 00:32:17,2022-01-01 00:49:23,1.0,3.97,14.5,3.66
"""id8""",2022-01-01 00:40:58,2022-01-01 01:00:59,4.0,8.44,25.5,0.0
"""id0""",2022-01-01 00:55:13,2022-01-01 01:25:49,1.0,12.61,37.5,12.39
"""id1""",2022-01-01 00:55:24,2022-01-01 01:00:45,1.0,1.49,6.5,0.0
…,…,…,…,…,…,…
"""id0""",2022-01-14 18:20:51,2022-01-14 18:34:09,1.0,1.07,9.0,0.0
"""id4""",2022-01-14 18:32:26,2022-01-14 18:50:36,2.0,5.57,18.0,5.58
"""id2""",2022-01-14 18:34:11,2022-01-14 18:39:18,3.0,0.92,5.5,2.45


Randomly split `df_nyc` into a train and test `DataFrames`

Create `X_train` with:
- `VendorID` as a categorical variable
- `trip_minutes` as the duration of the trip in minutes
- `passenger_count`,`trip_distance`,`fare_amount` as numerical features

and `y_train` with `tip_amount` as the target vector

Instantiate and fit the gradient boosting model

Make a `DataFrame` with `actual` and `pred` columns on the training data

Make a scatter plot of the `actual` versus `pred`

Get the root-mean-squared error of the prediction of the tip amount

Create the test feature matrix and target vector

Make predictions on the test `DataFrame` and make a scatterplot

Get the root-mean-squared error on the test data

### Exercise two
We now fit models and make predictions with a `Pipeline` approach.

Begin by creating `df_nyc_train` and `df_nyc_test` by:
- creating any feature columns and
- splitting `df_nyc`

Now create the train/test feature matrixes and target vectors

Create a pipeline for the numerical features by:
- imputing missing values with the median
- scaling values by the min/max of that column

Create the categorical `VendorID` feature

Make a `ColumnTransformer` to preprocess all features

Make a `Pipeline` to preprocess the features and do **linear regression**

Make a `DataFrame` of actual and predicted values on the **test** data

Get the RMSE of the model on the test data

## Solutions

### Solution to exercise one
We want to predict the tip amount in the NYC taxi database from the other columns

In [20]:
taxi_data = "../data/nyc_trip_data_1k.csv"
df_nyc = pl.read_csv(taxi_data,try_parse_dates=True)
df_nyc

VendorID,pickup,dropoff,passenger_count,trip_distance,fare_amount,tip_amount
str,datetime[μs],datetime[μs],f64,f64,f64,f64
"""id1""",2022-01-01 00:04:14,2022-01-01 00:26:12,1.0,10.83,31.0,0.0
"""id2""",2022-01-01 00:32:17,2022-01-01 00:49:23,1.0,3.97,14.5,3.66
"""id8""",2022-01-01 00:40:58,2022-01-01 01:00:59,4.0,8.44,25.5,0.0
"""id0""",2022-01-01 00:55:13,2022-01-01 01:25:49,1.0,12.61,37.5,12.39
"""id1""",2022-01-01 00:55:24,2022-01-01 01:00:45,1.0,1.49,6.5,0.0
…,…,…,…,…,…,…
"""id0""",2022-01-14 18:20:51,2022-01-14 18:34:09,1.0,1.07,9.0,0.0
"""id4""",2022-01-14 18:32:26,2022-01-14 18:50:36,2.0,5.57,18.0,5.58
"""id2""",2022-01-14 18:34:11,2022-01-14 18:39:18,3.0,0.92,5.5,2.45


Randomly split `df_nyc` into a train and test `DataFrames`

In [21]:
df_train,df_test = train_test_split(df_nyc,test_size=0.2,random_state=0)

Create `X_train` with:
- `VendorID` as a categorical variable
- `trip_minutes` as the duration of the trip in minutes
- `passenger_count`,`trip_distance`,`fare_amount` as numerical features

and `y_train` with `tip_amount` as the target vector

In [22]:
X_train = (
    df_train
    .with_columns(
        pl.col("VendorID").cast(pl.Categorical),
        trip_minutes = (pl.col("dropoff") - pl.col("pickup")).dt.total_minutes(),
    )
    .drop("pickup","dropoff","tip_amount")
)
y_train = df_train["tip_amount"]
X_train.head()

VendorID,passenger_count,trip_distance,fare_amount,trip_minutes
cat,f64,f64,f64,i64
"""id8""",1.0,0.66,5.0,4
"""id4""",2.0,2.86,9.5,8
"""id6""",1.0,11.6,51.2,35
"""id5""",1.0,1.71,9.5,11
"""id7""",6.0,2.62,10.0,9


Instantiate and fit the gradient boosting model

In [23]:
model = HistGradientBoostingRegressor(categorical_features="from_dtype")

model.fit(X_train,y_train)

HistGradientBoostingRegressor(categorical_features='from_dtype')

Make a `DataFrame` with `actual` and `pred` columns on the training data

In [24]:
train_pred_df = pl.DataFrame(
    {
        "actual":y_train,
        "pred":model.predict(X_train)
    }
)

Make a scatter plot of the `actual` versus `pred`

In [25]:
train_pred_df.plot.scatter(
    x="pred",
    y="actual",
    aspect='equal',
    width=200
    )

:Scatter   [pred]   (actual)

Get the root-mean-squared error of the prediction of the tip amount

In [26]:
rmse = root_mean_squared_error(train_pred_df["actual"], train_pred_df["pred"])

print(f"RMSE: {rmse}")

RMSE: 1.7323741349256456


Create the test feature matrix and target vector

In [27]:
X_test = (
    df_test
    .with_columns(
        pl.col("VendorID").cast(pl.Categorical),
        trip_minutes = (pl.col("dropoff") - pl.col("pickup")).dt.total_minutes(),
    )
    .drop("pickup","dropoff","tip_amount")
)
y_test = df_test["tip_amount"]

Make predictions on the test `DataFrame` and make a scatterplot

In [28]:
test_pred_df = pl.DataFrame(
    {
        "label":y_test,
        "pred":model.predict(X_test)
    }
)
test_pred_df.plot.scatter(
    x="pred",
    y="label",
    aspect='equal',
    width=200
    )

:Scatter   [pred]   (label)

Get the root-mean-squared error on the test data

In [29]:
rmse = root_mean_squared_error(test_pred_df["label"], test_pred_df["pred"])

print(f"RMSE: {rmse}")

RMSE: 2.6262628913585466


### Solution to exercise 2
We now fit models and make predictions with a `Pipeline` approach.

Begin by creating `df_nyc_train` and `df_nyc_test` by:
- creating any feature columns and
- splitting `df_nyc`

In [30]:
df_nyc_train, df_nyc_test = train_test_split(
    df_nyc.select(
        "VendorID",
        (pl.col("dropoff") - pl.col("pickup")).dt.total_minutes().alias("trip_minutes"),
        "passenger_count",
        "trip_distance",
        "fare_amount",
        "tip_amount"
    ),
    test_size=0.2, 
    random_state=0
)

Now create the train/test feature matrixes and target vectors

In [31]:
X_train,y_train = df_nyc_train.drop("tip_amount"),df_nyc_train["tip_amount"]
X_test,y_test = df_nyc_test.drop("tip_amount"),df_nyc_test["tip_amount"]

Create a pipeline for the numerical features by:
- imputing missing values with the median
- scaling values by the min/max of that column

In [32]:
numeric_features = ["trip_minutes", "passenger_count","trip_distance","fare_amount",]
from sklearn.preprocessing import MinMaxScaler
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")), 
        ("scaler", MinMaxScaler())
    ]
)

Create the categorical `VendorID` feature

In [33]:
categorical_features = ["VendorID"]
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore",sparse_output=False)),
    ]
)

Make a `ColumnTransformer` to preprocess all features

In [34]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ],
)

Make a `Pipeline` to preprocess the features and do **linear regression**

In [35]:
preprocess_model_pipeline = Pipeline(
    steps=[("preprocessor", preprocessor), ("model", LinearRegression())]
)
preprocess_model_pipeline.set_output(transform="polars")

preprocess_model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['trip_minutes',
                                                   'passenger_count',
                                                   'trip_distance',
                                                   'fare_amount']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['VendorID'])])),
                ('model', LinearRegression())])

Make a `DataFrame` of actual and predicted values on the **test** data

In [36]:
pred_df = pl.DataFrame(
    {
        "actual":y_test,
        "pred":preprocess_model_pipeline.predict(X_test)
    }
)
pred_df

actual,pred
f64,f64
1.0,1.984375
0.0,2.484375
3.56,1.609375
2.36,0.953125
2.66,1.140625
…,…
12.62,7.515625
0.0,1.34375
2.76,2.15625


Get the RMSE of the model on the test data

In [37]:
rmse = root_mean_squared_error(pred_df["label"], pred_df["pred"])

print(f"RMSE: {rmse}")

ColumnNotFoundError: label